### Find start/end times of SessionID from GSM

*import libraries*

In [1]:
import os
import sys
import sqlite3 as sl

import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
%matplotlib inline


In [2]:
pd.options.display.max_rows = 100

In [3]:
start_codes=datetime.now()

**import df from previously saved query**

In [4]:
## read .dat file if df is saved
df = pd.read_csv('./data/sql_result_onlyP006.txt')
df_targets = pd.read_csv('./data/P006_targets.csv')

**patient dictionary**

In [5]:
user_id_dict = {
    'TTSH-CHISELRGgLPwL3Lk6JTgBaVP4bjg==':'Test_01',
    'TTSH-CHISEL2gpumVcbJE267TKKceWbOA==':'NULL', 
    'TTSH-CHISEL0Pa/Gtc3DUqoPVREp+I/qg==':'001', 
    'TTSH-CHISEL9GsWU5zb50CfPyoLNy7qvg==':'002',
    'TTSH-CHISELamITRYYb9UymNH+3xzylgg==':'003', 
    'TTSH-CHISEL6IRO4pIMDUCpIRFbGnO26w==':'004',
    'TTSH-CHISELZSJj4EGZh0iiozqhx3Gf4g==':'005',
    'TTSH-CHISELCn5PqGNyMkKoyFYaOryw5g==':'006'
}; 


In [6]:
# replace PatientID with username
df.replace({'PatientID':user_id_dict},inplace=True)
df_targets.replace({'PatientID':user_id_dict},inplace=True)

In [7]:
df.head()

,PatientID,SessionID,GameID,TimeStampDate,TimeStampMS,VisualTargetXCoord,VisualTargetYCoord,TargetXCoord,TargetYCoord,XCoordinate,YCoordinate,KxGain,KyGain,KxyGain,VelocityX,VelocityY,CurrentSenseLeft,CurrentSenseRight,TotalForce,Sparc
0,006,TTSH-CHISELCJytTGjHB0mo9zS9wUD+mA==,2,2022-05-13 12:51:26,4311827,159.0,247.0,0.0,0.0,0.161025,0.236725,0.0,0.0,0.0,0.000000,0.000000e+00,-0.0496,-0.024,0.000000,0.0
1,006,TTSH-CHISELCJytTGjHB0mo9zS9wUD+mA==,2,2022-05-13 12:51:26,4311842,159.0,247.0,0.0,0.0,0.160975,0.236725,0.0,0.0,0.0,-0.002514,-7.629395e-07,-0.0176,0.024,0.348561,0.0
2,006,TTSH-CHISELCJytTGjHB0mo9zS9wUD+mA==,2,2022-05-13 12:51:26,4311852,159.0,247.0,0.0,0.0,0.160975,0.236725,0.0,0.0,0.0,-0.002514,-7.629395e-07,-0.0176,0.024,0.348561,0.0
3,006,TTSH-CHISELCJytTGjHB0mo9zS9wUD+mA==,2,2022-05-13 12:51:26,4311862,159.0,247.0,0.0,0.0,0.160925,0.236725,0.0,0.0,0.0,-0.002513,1.907349e-07,-0.0176,0.024,0.348482,0.0
4,006,TTSH-CHISELCJytTGjHB0mo9zS9wUD+mA==,2,2022-05-13 12:51:26,4311872,159.0,247.0,0.0,0.0,0.160925,0.236725,0.0,0.0,0.0,-0.002513,1.907349e-07,-0.0176,0.024,0.348482,0.0


In [8]:
df_targets.head()

,PatientID,CreatedDate,SessionID,Score,Strength,Coordination,Agility,SessionTime,NumberOfTargets
0,006,2022-05-13 04:51:43,TTSH-CHISELCJytTGjHB0mo9zS9wUD+mA==,22.0,90.000000,47.782007,36.587932,0.105858,3
1,006,2022-05-13 04:54:31,TTSH-CHISEL49l6A4YNIUKlgrfpd81V6Q==,4.0,-1.000000,82.718265,30.927633,0.146582,1
2,006,2022-05-13 10:23:22,TTSH-CHISELugL3DxELE0eZjcBnurRzwA==,3912.0,76.429504,45.177369,13.062145,15.236070,461
3,006,2022-05-14 10:43:42,TTSH-CHISELy0I7yzluAE23FF63m+IH6A==,4313.0,66.936707,42.048936,38.771131,31.116016,743
4,006,2022-05-14 12:27:32,TTSH-CHISELFZdp5pooS02/Lf7vPNdjqQ==,1776.0,66.448135,40.479520,31.291117,23.131092,471


In [9]:
# number of sessions from GSM 
len(df_targets)

34

In [10]:
# format date
df['date_time'] = pd.to_datetime(df['TimeStampDate'], format='%Y-%m-%d %H:%M:%S')
df_targets['date_time'] = pd.to_datetime(df_targets['CreatedDate'], format='%Y-%m-%d %H:%M:%S')


In [11]:
# remove SessionID when null
df = df[df['SessionID'].notna()]
df_targets = df_targets[df_targets['SessionID'].notna()]

In [12]:
# make copy
df1 = df.reset_index()

In [13]:
list_sessions = df1[df1['SessionID'].notna()]['SessionID'].unique()

In [14]:
# number of sessions in df (from TFDM)
len(list_sessions)

33

In [15]:
# extract start/end times of each session, after removal of last row
session_nr = []
start_time = []
end_time = []
for session in list_sessions:
    session_temp = df1[df1['SessionID']==session]
    
    # drop all last index, to make life easier
    last_index=session_temp['index'].iloc[-1]
    df1.drop(df1[(df1['index'] == last_index)].index,axis=0,inplace=True)
    
    session_nr.append(session)
    start_time.append(session_temp['date_time'].iloc[0])
    end_time.append(session_temp['date_time'].iloc[-1])

In [16]:
# put in df
time_session = []
for row in range(0,len(session_nr)):
    time_session.append(row)
time_session = pd.DataFrame(time_session,columns=['session_nr'])   
time_session['SessionID'] = session_nr
time_session['start'] = start_time
time_session['end'] = end_time


In [17]:
time_session

,session_nr,SessionID,start,end
0,0,TTSH-CHISELCJytTGjHB0mo9zS9wUD+mA==,2022-05-13 12:51:26,2022-05-13 12:51:32
1,1,TTSH-CHISEL49l6A4YNIUKlgrfpd81V6Q==,2022-05-13 12:54:12,2022-05-13 12:54:21
2,2,TTSH-CHISELugL3DxELE0eZjcBnurRzwA==,2022-05-13 18:07:33,2022-05-13 18:23:12
3,3,TTSH-CHISELy0I7yzluAE23FF63m+IH6A==,2022-05-14 18:09:40,2022-05-14 18:43:28
4,4,TTSH-CHISELFZdp5pooS02/Lf7vPNdjqQ==,2022-05-14 19:05:26,2022-05-14 19:29:26
5,5,TTSH-CHISELUZpVp0Dma0uZJIXh38s0Mw==,2022-05-14 20:28:44,2022-05-14 20:48:56
6,6,TTSH-CHISELqaY0g1flJEeOqfrdXVGQ/Q==,2022-05-15 07:48:15,2022-05-15 08:52:41
7,7,TTSH-CHISELHKnC3M5tTUGTUMKnDNBGUw==,2022-05-15 12:11:49,2022-05-15 12:45:41
8,8,TTSH-CHISELr/6lkn0sZEWvWPASnpFNbw==,2022-05-15 16:47:31,2022-05-15 17:18:00
9,9,TTSH-CHISELuqXAumyHq0m63LgRYS3Veg==,2022-05-16 10:35:02,2022-05-16 12:25:31


In [18]:
merged_df = pd.merge(df_targets,time_session,on='SessionID',how='outer')
merged_df

,PatientID,CreatedDate,SessionID,Score,Strength,Coordination,Agility,SessionTime,NumberOfTargets,date_time,session_nr,start,end
0,006,2022-05-13 04:51:43,TTSH-CHISELCJytTGjHB0mo9zS9wUD+mA==,22.0,90.000000,47.782007,36.587932,0.105858,3,2022-05-13 04:51:43,0.0,2022-05-13 12:51:26,2022-05-13 12:51:32
1,006,2022-05-13 04:54:31,TTSH-CHISEL49l6A4YNIUKlgrfpd81V6Q==,4.0,-1.000000,82.718265,30.927633,0.146582,1,2022-05-13 04:54:31,1.0,2022-05-13 12:54:12,2022-05-13 12:54:21
2,006,2022-05-13 10:23:22,TTSH-CHISELugL3DxELE0eZjcBnurRzwA==,3912.0,76.429504,45.177369,13.062145,15.236070,461,2022-05-13 10:23:22,2.0,2022-05-13 18:07:33,2022-05-13 18:23:12
3,006,2022-05-14 10:43:42,TTSH-CHISELy0I7yzluAE23FF63m+IH6A==,4313.0,66.936707,42.048936,38.771131,31.116016,743,2022-05-14 10:43:42,3.0,2022-05-14 18:09:40,2022-05-14 18:43:28
4,006,2022-05-14 12:27:32,TTSH-CHISELFZdp5pooS02/Lf7vPNdjqQ==,1776.0,66.448135,40.479520,31.291117,23.131092,471,2022-05-14 12:27:32,4.0,2022-05-14 19:05:26,2022-05-14 19:29:26
5,006,2022-05-14 12:55:52,TTSH-CHISELUZpVp0Dma0uZJIXh38s0Mw==,4900.0,76.430061,46.181676,27.955052,20.194975,583,2022-05-14 12:55:52,5.0,2022-05-14 20:28:44,2022-05-14 20:48:56
6,006,2022-05-15 00:53:56,TTSH-CHISELqaY0g1flJEeOqfrdXVGQ/Q==,3855.0,46.959461,39.042252,75.466805,46.760323,979,2022-05-15 00:53:56,6.0,2022-05-15 07:48:15,2022-05-15 08:52:41
7,006,2022-05-15 04:47:30,TTSH-CHISELHKnC3M5tTUGTUMKnDNBGUw==,4987.0,76.627426,45.945252,10.391109,20.661438,598,2022-05-15 04:47:30,7.0,2022-05-15 12:11:49,2022-05-15 12:45:41
8,006,2022-05-15 09:20:23,TTSH-CHISELr/6lkn0sZEWvWPASnpFNbw==,2047.0,45.560345,39.623017,63.148769,26.302641,507,2022-05-15 09:20:23,8.0,2022-05-15 16:47:31,2022-05-15 17:18:00
9,006,2022-05-16 04:26:33,TTSH-CHISELuqXAumyHq0m63LgRYS3Veg==,7846.0,66.348015,43.414947,31.420210,59.058811,1363,2022-05-16 04:26:33,9.0,2022-05-16 10:35:02,2022-05-16 12:25:31


In [19]:
# save df to csv
PatientID = df1['PatientID'].unique()[0]
merged_df.to_csv('./data/P' + PatientID + '_session_time.csv')

In [20]:
print(f'total run time = {datetime.now()-start_codes}')

total run time = 0:01:58.334207
